# Recurrent Neural Network (RNN, 순환 신경망)

<br>

numpy로 만든 RNN을 이용해 단어 생성을 수행한다.

<img src="http://corochann.com/wp-content/uploads/2017/05/text_sequence_predict.png" width="400" height="200">

<br>우린 주어진 문단에서 다음 단어를 어떻게 예측하도록 네트워크를 학습시키는지 볼것이다. 이 작업은 네트워크가 단어들의 시퀀스를 기억하고 난 뒤의 순환 아키텍쳐가 요구된다. 순서도 중요하다. 1000번 반복하면 발음 가능한 것을 얻을 수 있다. 트레이닝 시간을 길게 할수록 (더) 좋다. 어떤 문장 배열이든 넣을 수 있다. (단어들, 파이썬, HTML, 기타 등등)<br><br>

---------

<br>순환 네트워크(Recurrent Network)란 무엇인가?
--------------

피드포워드 네트워크(Feedforward network)는 입력과 출력간의 패턴을 배우는데 좋다.<br><br>

<img src="https://www.researchgate.net/profile/Sajad_Jafari3/publication/275334508/figure/fig1/AS:294618722783233@1447253985297/Fig-1-Schematic-of-the-multilayer-feed-forward-neural-network-proposed-to-model-the.png"><br>

<img src="https://s-media-cache-ak0.pinimg.com/236x/10/29/a9/1029a9a0534a768b4c4c2b5341bdd003--city-year-math-patterns.jpg"><br>

- 기온 & 위치
- 높이 & 무게
- 차의 속도와 브랜드<br><br><br>

그러나 데이터의 순서가 중요하다면 어떨까?<br><br>

<img src="http://www.aboutcurrency.com/images/university/fxvideocourse/google_chart.jpg" width="300" height="200"><br><br>

<img src="http://news.mit.edu/sites/mit.edu.newsoffice/files/styles/news_article_image_top_slideshow/public/images/2016/vondrick-machine-learning-behavior-algorithm-mit-csail_0.jpg?itok=ruGmLJm2" width="500" height="500"><br><br>


알파벳, 노래의 가사. 이것들은 조건부 기억(Conditional Memory)를 사용해 저장된다.  만약 이전 요소에 접근했다면, 오로지 하나의 요소에만 접근할 수 있다. (linked list - 자료구조 형태처럼)<br>

순환 네트워크로 들어가보자.<br><br>

이전 시간의 단계에서 다음 시간 단계의 네트워크로 돌아가 hidden state를 집어넣는다.<br>

<img src="https://iamtrask.github.io/img/basic_recurrence_singleton.png" width="400" height="200">

그래서 이런식으로 데이터의 실행이 일어나는 대신<br>

#### input -> hidden -> output<br><br>

이렇게 나타난다.<br>

#### (input + prev_hidden) -> hidden -> output<br><br>

잠깐, 왜 이런 형태가 아니지?<br>

#### (input + prev_input) -> hidden -> output<br><br>
 
배선된 입력 순환(input recurrence)은 즉시 이전의 데이터 포인트만 기억할 뿐인데도, 숨겨진 순환(hidden recurrence)은 무엇을 기억해야할지 배운다.

---------

**RNN : Intuition**<br><br>
- RNN은 80년대 시계열 모델링을 위해 나온 신경망 모델이다.<br>
- 네트워크의 구조는 피드포워드 네트워크(Feedforward network)와 비슷하며, 이전 시간에 의존해 각 시간마다 활성화되는 순환 hidden state를 사용한다는 점이 (다른 신경망과) 구별되는 부분이다.<br><br>

<img src="https://image.slidesharecdn.com/ferret-rnn-151211092908/95/recurrent-neural-networks-part-1-theory-10-638.jpg?cb=1449826311" width="500" height="300"><br>

<img src="https://www.mathworks.com/help/examples/nnet/win64/RefLayRecNetExample_01.png" width="500" height="300"><br><br>

**RNN 공식**<br>
<img src="https://cdn-images-1.medium.com/max/1440/0*TUFnE2arCrMrCvxH.png" width="400" height="200"><br>
기본적으로 현재 hidden state *h(t)*는 이전 hidden state *h(t-1)*와 현재 입력값 *x(t)*의 함수 *f* 라고 말할 수 있다. theta들은 함수 *f* 의 파라미터들이다. 네트워크는 일반적으로 *h(t)*를 사용해서 배우며, 일종의 *t* 의 입력값들의 과거 시퀀스 양상과 관련된 손실 요약본이라 볼 수 있다. <br>

<br><br>**손실 함수**<br>
<img src="https://cdn-images-1.medium.com/max/1440/0*ZsEG2aWfgqtk9Qk5." width="500" height="300"><br><br>

주어진 x값 시퀀스와 쌍인 y값의 시퀀스의 총 손실은, 모든 시간 단계별 손실의 합이다. 예시로, 만약 L(t)가 x(1)~x(t)가 주어졌을 때 y(t)의 negative log-likelihood라면, 그것들을 전부 더해서 시퀀스의 손실을 구할 수 있다.<br>

---------

## 단계<br>

- 가중치를 랜덤하게 초기화한다<br>
- 문자쌍을 모델에 넣는다 (입력 문자 & 목표 문자. 목표 문자는 네트워크가 추측해야 하는 시퀀스 내부의 다음 문자이다.)<br>
- 에러 측정 (이전 확률과 목표 문자 사이의 거리)<br>
- 손실에 각 파라미터가 끼치는 영향을 보기 위한 기울기 계산 (시간 관통 역전파)<br>
- 손실을 최소화하는 기울기를 통하는 방향으로 전체 파라미터 업데이트<br>
- 반복! 손실이 확실하게 small이 될때까지<br><br>

어떤 경우에 쓰이나?<br>
- 시계열 예측 (날씨 예보, 주가, 교통량 기타 등등)<br>
- 연속 데이터 생성 (음악, 비디오, 음향 기타 등등)<br><br>

다른 예시들<br>
-https://github.com/anujdutt9/RecurrentNeuralNetwork (binary addition)<br><br>

다음은?<br>
1 LSTM (Long Short-Term Memory) 네트워크<br>
2 양방향 네트워크 (Bidirectional networks)<br>
3 재귀 네트워크 (Recursive networks)<br>


------

코드의 네 부분<br>
------------------------
* 트레이닝 데이터 로딩<br>
 - 문자를 벡터로 인코딩<br>
* Recurrent Network (순환 네트워크) 정의<br>
* 손실 함수(loss function) 정의<br>
 - 포워드 과정<br>
 - 손실<br>
 - 백워드 과정<br>
* 모델로부터 문장을 만들기 위한 함수 정의<br>
* 네트워크 트레이닝<br>
 - 네트워크에 입력<br>
 - 기울기 계산과 모델 파라미터 업데이트<br>
 - 트레이닝의 향상성을 보기 위한 텍스트 출력<br><br>

--------------

트레이닝 데이터 로딩<br>
-----------------

네트워크는 입력값으로 큰 텍스트 파일이 필요하다. 파일에 담긴 콘텐츠는 네트워크를 트레이닝 시키는데에 사용될 것이다. 카프카의 변신을 사용했다(저작권에서 자유롭다) 카프카는 이상한 친구라서 좋으니까 ;) <br>

In [3]:
data = open('kafka.txt', 'r').read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print('data has %d chars, %d unique' % (data_size, vocab_size))

data has 137628 chars, 80 unique


-----------

인코딩/디코딩 문자/벡터<br>
-----------------------

신경망은 벡터로 동작하므로 (벡터는 float의 array다) 글자를 벡터로 encode / decode 할 방법이 필요하다.<br>고유한 글자들의 수를 셀건데 (vocabulary size) 이것이 벡터의 크기가 된다. 벡터는 문자의 위치(1이 있는)만 제외한 전부 0이다.<br><br>
첫번째로 *vocab size*를 계산해보자

In [4]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

{'-': 0, '3': 1, '9': 2, ':': 3, 'M': 4, 'I': 5, '@': 6, '(': 7, 'K': 8, '$': 9, 's': 10, 'J': 11, 'f': 12, 'a': 13, '.': 14, '"': 15, ' ': 16, ';': 17, 'Y': 18, 'l': 19, 'j': 20, 'C': 21, 'Q': 22, 'D': 23, 'S': 24, '\n': 25, 'L': 26, 'U': 27, 'N': 28, 'P': 29, '7': 30, 'R': 31, 'u': 32, 'c': 33, 'G': 34, 'x': 35, 'F': 36, '!': 37, '5': 38, '%': 39, 'i': 40, 'H': 41, 'r': 42, 'd': 43, '8': 44, 'X': 45, 'b': 46, 'w': 47, 'n': 48, 't': 49, 'h': 50, 'v': 51, "'": 52, 'p': 53, 'm': 54, '1': 55, '2': 56, '4': 57, 'V': 58, ')': 59, 'o': 60, '*': 61, 'E': 62, 'z': 63, '챌': 64, 'O': 65, 'g': 66, '?': 67, 'T': 68, '0': 69, 'q': 70, 'e': 71, '/': 72, 'A': 73, 'k': 74, 'B': 75, 'W': 76, '6': 77, ',': 78, 'y': 79}
{0: '-', 1: '3', 2: '9', 3: ':', 4: 'M', 5: 'I', 6: '@', 7: '(', 8: 'K', 9: '$', 10: 's', 11: 'J', 12: 'f', 13: 'a', 14: '.', 15: '"', 16: ' ', 17: ';', 18: 'Y', 19: 'l', 20: 'j', 21: 'C', 22: 'Q', 23: 'D', 24: 'S', 25: '\n', 26: 'L', 27: 'U', 28: 'N', 29: 'P', 30: '7', 31: 'R', 32: 'u',

<br><br>다음으로 문자를 숫자로 인코딩 디코딩할 딕셔너리 2개를 만든다.

마지막으로 이렇게 문자들로부터 벡터를 만든다: <br>
위에 정의된 딕셔너리는 256 대신 61 사이즈의 벡터를 만들 수 있게 해준다. <br>
여기 'a' 문자 예시를 보자. 벡터에는 char_to_ix['a']에 1이 들어가는 위치 외에는 0만 있다.

In [5]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print(vector_for_char_a.ravel())

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]


--------------

네트워크 정의<br>
------------------

신경망은 3개의 레이어로 구성된다.<br>
- 입력 레이어<br>
- 히든 레이어<br>
- 출력 레이어<br><br>

모든 레이어는 다음으로 완전 연결(fully connected)이 된다 : 한 레이어의 각 노드는 다음 레이어의 전체 노드로 연결된다.<br>
모든 히든 레이어는 아웃풋과 그 자신으로 연결되어 있다 : 반복(iteration)으로부터 얻어진 값은 다음 반복에 사용된다.<br> 

값을 중앙화 하는 건 트레이닝에서 중요하다(*hyper parameters*) *시퀀스의 길이(sequence length)* 와 *학습률(learning rate)* 도 마찬가지로 정의한다.

In [6]:
#hyperparameters

In [7]:
#model parameters

hidden_size = 100
seq_length = 25
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

모델 파라미터들은 트레이닝을 하는 동안 맞춰진다.<br><br>
- *WxH* 는 벡터와 관련된 파라미터들이다. 하나의 입력값에서 히든 레이어 까지 담고 있다.<br>
- *Whh* 는 히든 레이어, 그 자신과 연결된 파라미터들이다. RNN의 키(Key)이다 : hidden state의 이전 값들로부터, 다음 반복(iteration)에 자기자신으로 다시 들어가는 것으로 순환이 된다.<br>
- *Why*는 히든 레이어에서 출력으로 연결된 파라미터들이다.<br><br>
- *bh*는 히든 바이어스 (hidden bias)를 갖고 있다.<br>
- *by*는 출력 바이어스 (output bias)를 갖고 있다.<br><br>

다음 섹션에서 이 파라미터들이 문장을 만드는데 어떻게 사용되는지 보자.<br>

---------------------

손실 함수 정의<br>
---------------------------------------------

모든 신경망을 트레이닝시키는 것이 손실(Loss)의 핵심 컨셉이다. 얼마나 우리의 모델이 좋은지 알려준다. 손실을 줄일수록, 더 나은 모델이다. (좋은 모델은 트레이닝 출력값에 근접한 예측 값을 내놓는 모델이다.) 트레이닝을 하는 동안 우리는 손실을 줄이고 싶다.<br><br>

손실 함수는 손실과 함께 기울기를 계산한다 (백워드 과정을 봐라)<br>
* 포워드 과정을 수행한다 : 트레이닝 세트로부터 주어진 문자의 다음 문자를 계산한다<br>
* 예측한 문자와 목표 문자를 비교해서 손실을 계산한다 (트레이닝 셋에서 목표 문자는 이어지는 문자의 입력이다)<br>
* 기울기 연산을 위해 백워드 과정을 계산한다<br><br>

이 함수가 받는 입력: <br>
* 입력 글자 리스트<br>
* 목표 글자 리스트<br>
* 이전 hidden state<br><br>

이 함수의 출력들:<br>
* 손실<br>
* 레이어 사이의 각 파라미터를 위한 기울기<br>
* 마지막 hidden state<br><br>



--------------

포워드 과정<br>
----------------

포워드 과정(foward pass)은 트레이닝 세트로부터 주어진 문자의 다음 문자 계산을 위해, 모델의 파라미터(Wxh, Whh, Why, bh, by)를 사용한다<br><br>

xs[t]는 t 위치에 있는 문자 인코딩 벡터<br>
ps[t]는 다음 문자의 확률들<br><br>

<img src="https://deeplearning4j.org/img/recurrent_equation.png" width="400" height="300"><br><br>

```python
hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
```
<br><br><br>
또는 각 문자별 의사 코드<br><br>
```python
hs = input * Wxh + last_value_of_hidden_state * Whh + bh
ys = hs * Why + by
ps = normalized(ys)
```
<br>

-----------------------

백워드 과정(Backward pass)<br>
-------------------

모든 기울기를 계산하는 단순한 방법은 각 파라미터별 작은 변화의 손실을 재계산하는 것이며, 할 수는 있지만 시간을 잡아먹는다. 모든 파라미터별 모든 기울기를 한 번에 계산하는 테크닉이 있다 : backdrop 전파이다. 기울기를 포워드 과정(forward pass)의 반대 방향으로 계산하는, 간단한 테크닉을 쓴다.<br><br>

목표는 forward 공식을 위한 기울기 계산<br><br>
```python
hs = input*Wxh + last_value_of_hidden_state*Whh + bh<br>
ys = hs*Why + by
```
<br><br>

한 데이터 포인트에서의 손실<br>
<img src="http://i.imgur.com/LlIMvek.png" width="400" height="300"><br>

손실을 줄여나가는 f(함수) 내부의 스코어 변화를 어떻게 계산할 수 있을까? 이를 위해 기울기(gradient)가 필요하다.<br><br>

모든 출력 유닛들이 각 히든 유닛 에러에 영향을 주므로, 우리는 시퀀스에서 각 시간 단계별 계산된 기울기를 모두 더하고 파라미터를 업데이트하는데에 사용한다. 따라서 파라미터 기울기는 :<br><br><br>

<img src="http://i.imgur.com/Ig9WGqP.png" width="400" height="300"><br><br>

손실의 첫번째 기울기. 연쇄법칙(chain rule)을 통해 역전파(backpropagate)해보자.<br> 
<img src="http://i.imgur.com/SOJcNLg.png" width="400" height="300"><br><br>

연쇄법칙은 합성 함수(composite function)의 도함수(derivative)를 찾아내는 방법이다. 또는 하나 또는 그 이상의 함수들의 결합으로 함수들을 만든다.<br>
<img src="http://i.imgur.com/3Z2Rfdi.png" width="400" height="300"><br><br>
<img src="http://mathpullzone-8231.kxcdn.com/wp-content/uploads/thechainrule-image3.jpg" width="400" height="300"><br><br>
<img src="https://i0.wp.com/www.mathbootcamps.com/wp-content/uploads/thechainrule-image1.jpg?w=900" width="400" height="300"><br><br>

In [8]:
def lossFun(inputs, targets, hprev):
    xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass                                                                                                                                                                              
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
        xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
        
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        # output probabilities
        dy = np.copy(ps[t])
        # derive our first gradient
        dy[targets[t]] -= 1 # backprop into y  
        # compute output gradient -  output times hidden states transpose
        # When we apply the transpose weight matrix,  
        # we can think intuitively of this as moving the error backward
        # through the network, giving us some sort of measure of the error 
        # at the output of the lth layer. 
        # output gradient
        dWhy += np.dot(dy, hs[t].T)
        # derivative of output bias
        dby += dy
        # backpropagate!
        dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
        dbh += dhraw # derivative of hidden bias
        dWxh += np.dot(dhraw, xs[t].T) # derivative of input to hidden layer weight
        dWhh += np.dot(dhraw, hs[t-1].T) # derivative of hidden layer to hidden layer weight
        dhnext = np.dot(Whh.T, dhraw) 
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

모델로부터 문장 생성
----------------

In [9]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
    """                                                                                                                                                                                         
    sample a sequence of integers from the model                                                                                                                                                
    h is memory state, seed_ix is seed letter for first time step   
    n is how many characters to predict
    """
    #create vector
    x = np.zeros((vocab_size, 1))
    # customize it for our seed char
    x[seed_ix] = 1
    # list to store generated chars
    ixes = []
    # for as many characters as we want to generate
    for t in range(n):
        #a hidden state at a given time step is a function 
        #of the input at the same time step modified by a weight matrix 
        #added to the hidden state of the previous time step 
        #multiplied by its own hidden state to hidden state matrix.
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        #compute output (unnormalised)
        y = np.dot(Why, h) + by
        ## probabilities for next chars
        p = np.exp(y) / np.sum(np.exp(y))
        #pick one with the highest probability 
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        #create a vector
        x = np.zeros((vocab_size, 1))
        #customize it for the predicted char
        x[ix] = 1
        #add it to the list
        ixes.append(ix)

    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
# predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

----
 )d3FwQ65
;;C97j3u2ISzcIVBjEUpR

Qi*adlv7t-jyfPPdm poHJ2)GrYoT챌J"챌eva''.6w"aG9HU%2,0A,:Vr(W,)UX'o8iw,-R Q1.. "eoHgdMea2mv73JN
s)J8i$P.xp(gMJV$Uxb,wehJQ,PUAVj,I8kS2,X3I9Qk3TmUMk,"D$9LXn3E%1p0hbq8Y,D2Tqp 
----


트레이닝<br>
-------------

코드의 마지막 부분은 트레이닝 loop의 메인이다 : <br>
* 네트워크에 파일의 일부를 넣는다. 파일 일부의 사이즈는 *seq_length*다.<br>
* 사용하는 손실 함수는:<br>
- 주어진 입력/출력 쌍을 위한 모델의 모든 파라미터들을 forward pass 에서 계산한다<br>
- 모든 기울기를 backward pass에서 계산한다.<br>
* 네트워크의 파라미터들을 사용해, 랜덤한 seed로부터 문장을 출력한다.<br>
* Adagrad (Adaptive Gradient) 테크닉을 사용해 모델을 업데이트한다.<br><br><br>


**입력값과 목표값을 손실함수에 넣기**<br>

데이터 파일로부터 문자 어레이를 두개 만들고, 타겟 어레이는 움직이며 입력 어레이와 비교한다.
입력 어레이의 각 문자마다, 타겟 어레이는 이어지는 문자를 내놓는다.<br>

In [10]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print("targets", targets)

inputs [65, 48, 71, 16, 54, 60, 42, 48, 40, 48, 66, 78, 16, 47, 50, 71, 48, 16, 34, 42, 71, 66, 60, 42, 16]
targets [48, 71, 16, 54, 60, 42, 48, 40, 48, 66, 78, 16, 47, 50, 71, 48, 16, 34, 42, 71, 66, 60, 42, 16, 24]


-----------------------

**파라미터 업데이트를 위한 Adagrad**<br><br>

경사 하강법(gradient descent)의 일종이다.

<img src="http://www.logos.t.u-tokyo.ac.jp/~hassy/deep_learning/adagrad/adagrad2.png"><br>


스텝 사이즈 = 학습률(learning rate)<br>

모델의 파라미터를 업데이트하는 가장 쉬운 테크닉은 이거다:<br>

```python
param += dparam * step_size
```
<br><br>

Adagrad는 트레이닝동안 step_size를 줄여나가는 더 효율적인 테크닉이다<br><br>
시간마다 증가하는 메모리 변수를 사용하고:<br>
```python
mem += dparam * dparam
```
<br><br>

step_size를 계산하기 위해 이를 사용한다<br>
```python
step_size = 1./np.sqrt(mem + 1e-8)
```
<br><br>

요약: <br>
```python
mem += dparam * dparam
param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update 
```
<br><br>

**Smooth_loss**<br>

smooth_loss는 트레이닝에서 어떤 일도 하지 않는다. 손실의 low pass filtered version이다. <br>
```python
smooth_loss = smooth_loss * 0.999 + loss * 0.001
```
<br>

마지막 반복의 손실 평균을 구하는 방법이 진척도를 추적하기 더 좋다.

----------------

마지막으로<br>
---------------

아래는 트레이닝과 시간마다 텍스트 생성을 함께 수행하는 main loop 코드이다.<br>

In [11]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    # check "How to feed the loss function to see how this part works
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # forward seq_length characters through the net and fetch gradient                                                                                                                          
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # sample from the model now and then                                                                                                                                                        
    if n % 1000 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
        sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

    p += seq_length # move data pointer                                                                                                                                                         
    n += 1 # iteration counter

iter 0, loss: 109.550658
----
 O%nPLacKsv)YF-%bng.G1c9-@5$2Vu
p4DX(*5ndR1 Y.de saOP7Y1) 'IRpMTVf
lY RE(ekw%E;7
uyT0q7F?*챌aO,Kb(PRs0qc1"C@dJTUyzhyMsyHKwv.QuAaV)"BQ;K:@KoO챌Gb(QL34R@q3.;E0!vk2sVLGnr?H' SFig1w.K,Sl)h)lYs!01dMgbJeE7y(/t 
----
iter 1000, loss: 41.878356
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 2000, loss: 15.419662
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 3000, loss: 5.681937
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 4000, loss: 2.097729
----
 ne morning,

iter 35000, loss: 0.001226
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 36000, loss: 0.001190
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 37000, loss: 0.001155
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 38000, loss: 0.001123
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 39000, loss: 0.001092
----
 ne morn

iter 70000, loss: 0.000589
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 71000, loss: 0.000581
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 72000, loss: 0.000572
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 73000, loss: 0.000564
----
 ne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor Sne morning, when Gregor S 
----
iter 74000, loss: 0.000556
----
 ne morn